![title](DATA/birds-nest-4-eggs.png)

###### image:https://www.publicdomainpictures.net/nl/view-image.php?image=61631&picture=vogels-nest-4-eieren

#   The effect of precipitation between march and july on the breeding of birds in the Netherlands 1990-2021 in changes compared to the year before.


Sources:

Dutch bird breeding per season (index):
https://cdn.knmi.nl/knmi/map/page/klimatologie/gegevens/maandgegevens/mndgeg_260_rh24.txt

Precipitation at the Bilt:
https://opendata.cbs.nl/statline/#/CBS/nl/dataset/84498NED/table?ts=1673294982549

In [1]:
#importing all necessary libraries
import pandas as pd
import functions_final_assignment as fn

In [2]:
#loading in the files with a yaml config file
config = fn.yaml_config()
print(config["precipitation"])

DATA/mndgeg_260_rh24.txt
